# Klasifikasi Makanan Indonesia


## Import & dependensi
Bagian ini mengimpor semua library dan modul yang dipakai sepanjang skrip: PyTorch (core, nn, optim, DataLoader), numpy, metrik dari sklearn, utilitas time dan os, serta arsitektur pretrained MNASNet0.5 dari torchvision. Juga mengimpor modul kustom MakananIndo (dataset) dan check_set_gpu() (utility untuk memilih device GPU/CPU). Ini menyiapkan building block untuk load data, membangun model, training, dan evaluasi.

## Pembuatan encoder label (create_label_encoder)
Fungsi ini mengiterasi seluruh dataset (for i in range(len(dataset))) dan memanggil dataset[i] untuk mengambil label (diasumsikan berbentuk string). Dari semua label dikumpulkan kelas unik, diurutkan, lalu dibuat dua mapping: label_to_idx (label string → integer index) dan idx_to_label (index → label string). Fungsi mengembalikan ketiga objek (label_to_idx, idx_to_label, unique_labels) yang dipakai untuk mengonversi label tekstual menjadi indeks angka saat training/validasi.

## Inisialisasi model MNASNet (create_mnasnet_model)
Fungsi ini memuat MNASNet0.5 dengan bobot pretrained (MNASNet0_5_Weights.IMAGENET1K_V1). Ia mencetak informasi model dan classifier awal, lalu—jika freeze_backbone=True—membekukan semua parameter model dengan param.requires_grad = False sehingga hanya layer akhir yang akan dilatih. Layer classifier diganti: diambil in_features dari elemen terakhir model.classifier lalu diganti dengan nn.Sequential(nn.Linear(in_features, num_classes), nn.Softmax(dim=1)). Setelah itu parameter kelas baru di-set requires_grad=True. Fungsi mengembalikan model yang sudah dimodifikasi.

# Model Testing and Prediction Generation

This notebook will:
1. Load the best trained MNASNet model
2. Process test images
3. Generate predictions
4. Save results to jawaban.csv

## 1. Import Libraries dan Juga Load Modelsnya

In [1]:
import torch
import torch.nn as nn
from torchvision.models import mnasnet0_5, MNASNet0_5_Weights
from torchvision import transforms
import pandas as pd
import os
from PIL import Image

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define class labels
idx_to_label = {
    0: 'bakso',
    1: 'gado_gado',
    2: 'nasi_goreng',
    3: 'rendang',
    4: 'soto_ayam'
}

# Create model
def create_mnasnet_model(num_classes):
    weights = MNASNet0_5_Weights.IMAGENET1K_V1
    model = mnasnet0_5(weights=weights)
    
    # Replace the final classification layer
    in_features = model.classifier[-1].in_features
    model.classifier = nn.Sequential(
        nn.Linear(in_features, num_classes),
        nn.Softmax(dim=1)
    )
    return model

# Initialize model
model = create_mnasnet_model(num_classes=5)

# Load trained weights
model.load_state_dict(torch.load('best_mnasnet_model.pth'))
model = model.to(device)
model.eval()

print("Model loaded successfully!")

Using device: cuda


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_31484\2141570886.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_mnasnet_model.pth')

Model loaded successfully!


## 2. Load and Process Test Data

In [2]:
# Read test.csv to get file list
test_df = pd.read_csv('test.csv')
print(f"Loaded {len(test_df)} test images")

# Print first few entries to verify
print("\nFirst few entries from test.csv:")
print(test_df.head())

Loaded 277 test images

First few entries from test.csv:
   filename  label
0  0001.jpg    NaN
1  0002.jpg    NaN
2  0003.jpg    NaN
3  0004.jpg    NaN
4  0005.jpg    NaN


## 3. Make Predictions

In [3]:
# Function to get prediction
def get_prediction(image_path):
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
    
    return idx_to_label[predicted.item()]

# Make predictions for all test images
predictions = []

print("Making predictions...")
for idx, row in test_df.iterrows():
    image_path = os.path.join('test', row['filename'])
    pred_label = get_prediction(image_path)
    predictions.append(pred_label)
    
    if (idx + 1) % 10 == 0:
        print(f"Processed {idx + 1}/{len(test_df)} images")

Making predictions...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Lenovo\\Documents\\Semester 7\\DeepLearn\\test\\0001.jpg'

## 4. Save Results

In [ ]:
# Create DataFrame with predictions
results_df = pd.DataFrame({
    'filename': test_df['filename'],
    'label': predictions
})

# Save to CSV
results_df.to_csv('jawaban.csv', index=False)
print("\nPredictions saved to jawaban.csv")
print("\nFirst few predictions:")
print(results_df.head())